In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-130776")
exp = Experiment(workspace=ws, name="Hyperdrive_run")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
                                {
                                    "--C": choice(1,2,3,4), "--max_iter": choice(75,100,125,150)
                                }
                            )

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.3, delay_evaluation=4)

# Code below makes a new directory for training and copies the train script
if "training" not in os.listdir():
    os.mkdir("./training")

#copying train.py to training directory    
import shutil
shutil.copy('train.py', './training')
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', compute_target=aml_compute, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyper_config = HyperDriveConfig(estimator=est, policy=policy, primary_metric_name="Accuracy",
                                hyperparameter_sampling=ps,
                                max_total_runs=20,
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyper_run = exp.submit(hyper_config)
RunDetails(hyper_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyper_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

filename = 'outputs/model.joblib'
joblib.dump(best_run.get_file_names(), filename)


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

In [ ]:
#Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='HyperDrive_run_Best_Model', model_path='outputs/',
                               properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})

### Hyperparameter tuning using AutoML

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(data)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [ ]:
# Adding y data coming from clean_data function to make the complete dataset
x['y'] = y

#label
label = "y"

In [ ]:
# Retrieving default datastore from workspace
default_datastore = ws.get_default_datastore() 

In [ ]:
# Register the dataset with name 'data AutoML'
# Uploads dataset to dataAutoML folder in default datastore
# Note: register_pandas_dataframe is an experimental method
dataSet = TabularDatasetFactory.register_pandas_dataframe(x, target=(default_datastore,'BankingData'),name='BankingData',show_progress=True)

In [ ]:
#Getting registered dataset from workspace for AutoML
train_data = ws.datasets.get('BankingData')

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    compute_target=aml_compute,
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, 'automl_bank_marketing')
ml_run = experiment.submit(automl_config)
RunDetails(ml_run).show()
ml_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

# Get the best run object
best_run, fitted_model = ml_run.get_output()
best_run_metrics = best_run.get_metrics()
    
# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                       properties={'Accuracy': best_run_metrics['accuracy']})

In [ ]:
# Register the model

from azureml.core import Model

# Printing models with accuracy
for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

In [ ]:
# Cluster clean up
aml_compute.delete()